cart başarılı ama overfittinge eğilimli çünkü tek bir karar ağacı aşırı uyum sağlayabilir bu sorunu çözen random forest algoritması var
bagging(bootstrap aggregating) yöntemi ile random subspace yöntemlerinin birleşiminden oluşuyor
ağaç oluşturulurken veri setinin 2/3 ü kullanılır kalan 1/3 lük veri ise performans değerlendirilmesinde kullanılır
örneğin bagging yöntemi:
1000 adet veri bulunsun elimizde rastgele seçilen 750 adeti bir ağaç haline getirilir = 1.ağaç
sonra yine 1000 veriden rastgele seçilen 750 adeti başka bir ağaç haline getirilir = 2.ağaç
... n adet ağaç en son birlikte yorumlanır

Her ağaç için seçilmeyen %33’lük veri, Out-of-Bag (OOB) verisi olarak adlandırılır.
Bu veriler, modelin performansını değerlendirmek için kullanılır.
Yani modelin içsel bir "cross-validation" yöntemi vardır.

In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

warnings.simplefilter(action='ignore', category=Warning)

df = pd.read_csv("datasets/diabetes.csv")

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [3]:
################################################
# Random Forests
################################################

rf_model = RandomForestClassifier(random_state=17)
rf_model.get_params()
# n_estimators parametresi fit edilmiş bağımsız ağaç sayısıdır

cv_results = cross_validate(rf_model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])
cv_results['test_accuracy'].mean() # 0.753896103896104
cv_results['test_f1'].mean() # 0.6190701534636385
cv_results['test_roc_auc'].mean() # 0.8233960113960114


np.float64(0.8233960113960114)

In [4]:
rf_params = {"max_depth": [5, 8, None], # bu sefer range ile girmiyoruz çünkü random forest bayağı zaman alıyor
             "max_features": [3, 5, 7, "auto"],
             "min_samples_split": [2, 5, 8, 15, 20],
             "n_estimators": [100, 200, 500]}


rf_best_grid = GridSearchCV(rf_model, rf_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

rf_best_grid.best_params_ # okuttuğum değerler içinde en iyi seçim bunlarmış

Fitting 5 folds for each of 180 candidates, totalling 900 fits


{'max_depth': None,
 'max_features': 5,
 'min_samples_split': 8,
 'n_estimators': 500}

In [5]:
rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X, y) # bunları modele entegre edelim

cv_results = cross_validate(rf_final, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])
cv_results['test_accuracy'].mean() # 0.76
cv_results['test_f1'].mean() # 0.64
cv_results['test_roc_auc'].mean() # 0.82
# ufacık farklarla iyileşme aldık

np.float64(0.8271054131054132)

In [ ]:
# yine istenirse bunlarda kontrol edilebilir

def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

plot_importance(rf_final, X)

def val_curve_params(model, X, y, param_name, param_range, scoring="roc_auc", cv=10):
    train_score, test_score = validation_curve(
        model, X=X, y=y, param_name=param_name, param_range=param_range, scoring=scoring, cv=cv)

    mean_train_score = np.mean(train_score, axis=1)
    mean_test_score = np.mean(test_score, axis=1)

    plt.plot(param_range, mean_train_score,
             label="Training Score", color='b')

    plt.plot(param_range, mean_test_score,
             label="Validation Score", color='g')

    plt.title(f"Validation Curve for {type(model).__name__}")
    plt.xlabel(f"Number of {param_name}")
    plt.ylabel(f"{scoring}")
    plt.tight_layout()
    plt.legend(loc='best')
    plt.show(block=True)

val_curve_params(rf_final, X, y, "max_depth", range(1, 11), scoring="roc_auc")